In [13]:
import pandas as pd
import numpy as np
import my_def

<h1>데이터확인 및 가공</h1>
2분기 뒤 타겟값 매칭

In [14]:
df = pd.read_csv('Merged_Sales_data_v2.6(shift1).csv')

In [15]:
col = ['년분기', '행정동', '업종별카테고리', '인구수', '지역생활인구', '장기외국인', '단기외국인',
       '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수', '버스정류장수',
       '월매출_shift1']
df = df[col]

In [16]:
df.columns = ['년분기', '행정동', '업종별카테고리', '인구수', '지역생활인구', '장기외국인', '단기외국인', '주차장면적(면)',
       '주차장개수(개소)', '학교수', '학생수', '버스정류장수', '월매출(점포)']

In [17]:
col = ['년분기', '인구수', '지역생활인구', '장기외국인', '단기외국인', '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수', '버스정류장수', '월매출(점포)']
df[col] = df[col].astype(int)

In [18]:
df = my_def.reduce_memory_usage(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69001 entries, 0 to 69000
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   년분기        69001 non-null  int16 
 1   행정동        69001 non-null  object
 2   업종별카테고리    69001 non-null  object
 3   인구수        69001 non-null  int32 
 4   지역생활인구     69001 non-null  int32 
 5   장기외국인      69001 non-null  int32 
 6   단기외국인      69001 non-null  int32 
 7   주차장면적(면)   69001 non-null  int32 
 8   주차장개수(개소)  69001 non-null  int16 
 9   학교수        69001 non-null  int16 
 10  학생수        69001 non-null  int16 
 11  버스정류장수     69001 non-null  int16 
 12  월매출(점포)    69001 non-null  int64 
dtypes: int16(5), int32(5), int64(1), object(2)
memory usage: 3.6+ MB
None


In [19]:
df[df["업종별카테고리"] == "기타"]

,년분기,행정동,업종별카테고리,인구수,지역생활인구,장기외국인,단기외국인,주차장면적(면),주차장개수(개소),학교수,학생수,버스정류장수,월매출(점포)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# '행정동'과 '업종별 카테고리' 컬럼 원핫 인코딩 수행
df2 = pd.get_dummies(df, columns=['행정동', '업종별카테고리'], drop_first=True)

# 피처와 타겟 설정
features = (
    ['년분기', '인구수', '지역생활인구', '장기외국인', '단기외국인', 
     '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수', '버스정류장수'] +
    [col for col in df2.columns if col.startswith('행정동_') or col.startswith('업종별카테고리_')]
)
target = '월매출(점포)'

X = df2[features]
y = df2[target]

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 선형 회귀 모델 학습 및 예측
model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# 모델 평가 호출
my_def.evaluate_model(model, X_train, X_test, y_train, y_test)


{'train_mse': np.float64(629687437939383.6),
 'test_mse': np.float64(5333348637214434.0),
 'train_rmse': np.float64(25093573.64),
 'test_rmse': np.float64(73029779.11),
 'train_r2': 0.99,
 'test_r2': 0.94,
 'train_mae': np.float64(11372172.38),
 'test_mae': np.float64(30829224.1),
 'train_mae_ratio': np.float64(5.29),
 'test_mae_ratio': np.float64(14.15)}

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# '행정동'과 '업종별 카테고리' 컬럼 원핫 인코딩 수행
df2 = pd.get_dummies(df, columns=['행정동', '업종별카테고리'], drop_first=True)

# 피처와 타겟 설정
features = (
    ['년분기', '인구수', '지역생활인구', '장기외국인', '단기외국인', 
     '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수', '버스정류장수'] +
    [col for col in df2.columns if col.startswith('행정동_') or col.startswith('업종별카테고리_')]
)
target = '월매출(점포)'

X = df2[features]
y = df2[target]

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 선형 회귀 모델 학습 및 예측
model = RandomForestRegressor(n_estimators= 300, n_jobs=-1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# 모델 평가 호출
my_def.evaluate_model(model, X_train, X_test, y_train, y_test)


KeyboardInterrupt: 

In [21]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# 랜덤포레스트 모델 초기화
rf = RandomForestRegressor(random_state=42)

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 200, 300],  # 트리 개수
    'max_depth': [None, 10, 20, 30],        # 최대 깊이
    'min_samples_split': [2, 5, 10],        # 노드를 분할하기 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4],          # 리프 노드의 최소 샘플 수
    'max_features': ['auto', 'sqrt', 'log2'], # 최대 피처 수
}

# GridSearchCV 초기화 (교차 검증 사용)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='r2')

# 학습 데이터에 대해 최적의 하이퍼파라미터 찾기
grid_search.fit(X_train, y_train)

# 최적의 파라미터와 최고 성능 출력
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best R2 Score: {grid_search.best_score_}")

# 최적의 모델로 예측 수행
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)

# 평가 결과 출력
from sklearn.metrics import r2_score, mean_squared_error

print(f"R2 Score on Test Set: {r2_score(y_test, y_pred)}")
print(f"RMSE on Test Set: {np.sqrt(mean_squared_error(y_test, y_pred))}")


Fitting 3 folds for each of 324 candidates, totalling 972 fits


KeyboardInterrupt: 